# Garmin CSV to Day One CSV

In [ ]:
import os
import numpy as np
import pandas as pd
import tkinter as tk
import datetime
from tkinter import filedialog
import ipywidgets as widgets

pd.options.mode.copy_on_write = True

### Import Garmin CSV

In [ ]:
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)
file_path_gar = filedialog.askopenfilename(
    title="Select Garmin CSV", initialdir=os.getcwd()
)
print("Selected file:")
print(file_path_gar)

### Import Strength level CSV

In [ ]:
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)
file_path_SL = filedialog.askopenfilename(
    title="Select Strength Level CSV", initialdir=os.getcwd()
)
print("Selected file:")
print(file_path_SL)

### Import dataframes

In [ ]:
df_gar = pd.read_csv(file_path_gar, sep=",", header=0, index_col=None)
df_SL = pd.read_csv(file_path_SL, sep=",", header=0, index_col=None)

### Summary statistics - Garmin

In [ ]:
df_gar["Datetime"] = pd.to_datetime(df_gar["Date"])

n_workouts = len(df_gar["Activity Type"])
print("Number of workouts: {}".format(n_workouts))

activity_types = np.unique(df_gar["Activity Type"])
n_activity_types = np.zeros(len(activity_types))
for i, activity_type in enumerate(activity_types):
    n_activity_types[i] = len(
        df_gar["Activity Type"][df_gar["Activity Type"] == activity_type]
    )

activity_type_sort_idx = np.flip(np.argsort(n_activity_types))
activity_types_sorted = activity_types[activity_type_sort_idx]
n_activity_types_sorted = n_activity_types[activity_type_sort_idx]

for i, activity_type in enumerate(activity_types_sorted):
    print(
        "     {}: {} {}".format(
            activity_type,
            int(n_activity_types_sorted[i]),
            ("activities" if int(n_activity_types_sorted[i]) > 1 else "activity"),
        )
    )

print(" ")

### Summary statistics - Strength Level

In [ ]:
df_SL["Datetime"] = pd.to_datetime(df_SL["Date Lifted"], dayfirst=False)

n_workouts = len(np.unique(df_SL["Date Lifted"]))
print("Total number of workouts: {}".format(n_workouts))

exercises = np.unique(df_SL["Exercise"])
n_exercises = len(exercises)
print("Number of different exercises performed: {}".format(n_exercises))

date_range = (df_SL["Datetime"].iloc[0] - df_SL["Datetime"].iloc[-1]).days
train_frequency = n_workouts / (date_range / 7)
print("Training frequency: {} workouts per week".format(np.round(train_frequency, 1)))
print(" ")

exercise_count = np.array(
    [
        len(np.unique(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]))
        for exercise in exercises
    ]
)
exercise_sortidx = np.flip(np.argsort(exercise_count))
exercises_sorted = exercises[exercise_sortidx]
exercise_count_sorted = exercise_count[exercise_sortidx]

exercise_sets = np.array(
    [len(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]) for exercise in exercises]
)
exercise_sets_sorted = exercise_sets[exercise_sortidx]

exercise_volume = np.array(
    [
        np.sum(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_volume_sorted = exercise_volume[exercise_sortidx]

exercise_record = np.array(
    [
        np.max(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_sorted = exercise_record[exercise_sortidx]

exercise_rep_record = np.array(
    [np.max(df_SL["Reps"][df_SL["Exercise"] == exercise]) for exercise in exercises]
)
exercise_rep_record_sorted = exercise_rep_record[exercise_sortidx]

exercise_record_bw = np.array(
        [
            (
                df_SL["Bodyweight (kg)"][
                    (df_SL["Exercise"] == exercise)
                    & (df_SL["Weight (kg)"] == exercise_record[i])
                ]
            ).min()
            for i, exercise in enumerate(exercises)
        ]
)
exercise_record_bw_sorted = exercise_record_bw[exercise_sortidx]

exercise_record_bwperc = (
    exercise_record
    / np.array(
        [
            (
                df_SL["Bodyweight (kg)"][
                    (df_SL["Exercise"] == exercise)
                    & (df_SL["Weight (kg)"] == exercise_record[i])
                ]
            ).min()
            for i, exercise in enumerate(exercises)
        ]
    )
    * 100
)
exercise_record_bwperc_sorted = exercise_record_bwperc[exercise_sortidx]

exercise_record_perc = np.array(
    [
        np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_perc_sorted = exercise_record_perc[exercise_sortidx]
weight_exercise_record_perc = [
    df_SL["Weight (kg)"].loc[
        (
            df_SL["Percentile (%)"]
            == np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        )
        & (df_SL["Exercise"] == exercise)
    ]
    for exercise in exercises
]
weight_exercise_record_perc = np.array(
    [item.values[0] if not item.empty else None for item in weight_exercise_record_perc]
)
weight_exercise_record_perc_sorted = weight_exercise_record_perc[exercise_sortidx]
reps_exercise_record_perc = [
    df_SL["Reps"].loc[
        (
            df_SL["Percentile (%)"]
            == np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        )
        & (df_SL["Exercise"] == exercise)
    ]
    for exercise in exercises
]
reps_exercise_record_perc = np.array(
    [item.values[0] if not item.empty else None for item in reps_exercise_record_perc]
)
reps_exercise_record_perc_sorted = reps_exercise_record_perc[exercise_sortidx]
bw_exercise_record_perc = [
    df_SL["Bodyweight (kg)"].loc[
        (
            df_SL["Percentile (%)"]
            == np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        )
        & (df_SL["Exercise"] == exercise)
    ]
    for exercise in exercises
]
bw_exercise_record_perc = np.array(
    [item.values[0] if not item.empty else None for item in bw_exercise_record_perc]
)
bw_exercise_record_perc_sorted = bw_exercise_record_perc[exercise_sortidx]


n_top = 10
print("Top {} most performed exercises: ".format(n_top))
for i, exercise in enumerate(exercises_sorted[:n_top]):
    if not np.isnan(exercise_record_sorted[i]):
        print(
            "     {}: \n        {} times, \n        {} kg max weight lifted @ {} kg ({}% bodyweight), \n        {} percentile best set ({} kg x {} @ {} kg bodyweight), \n        {} sets, \n        {} kg total volume".format(
                exercise,
                exercise_count_sorted[i],
                exercise_record_sorted[i],
                exercise_record_bw_sorted[i],
                np.round(exercise_record_bwperc_sorted[i], 1),
                exercise_record_perc_sorted[i],
                weight_exercise_record_perc_sorted[i],
                reps_exercise_record_perc_sorted[i],
                bw_exercise_record_perc_sorted[i],
                exercise_sets_sorted[i],
                np.round(exercise_volume_sorted[i], 1),
            )
        )
    else:
        print(
            "     {}: \n        {} times, \n        {} reps highest rep set (at bodyweight), \n        {} percentile best set, \n        {} sets".format(
                exercise,
                exercise_count_sorted[i],
                exercise_rep_record_sorted[i],
                exercise_record_perc_sorted[i],
                exercise_sets_sorted[i],
            )
        )

### Select start date

In [ ]:
dates_unique = pd.unique(df_gar["Date"])
dates_unique_str = np.array([date[:10] for date in dates_unique])

date_select = widgets.Select(
    options=dates_unique_str,
    value=dates_unique_str[0],
    description="Start date:",
    style={"description_width": "initial"},
)

confirm_button = widgets.Button(description="Confirm", disabled=False)
display(date_select, confirm_button)


def confirm_button_func(a):
    global start_date_str
    start_date_str = date_select.value

    print("Selected start date: " + start_date_str)
    return start_date_str


confirm_button.on_click(confirm_button_func)

### Rearrange dataframes

In [ ]:
start_date = df_gar.loc[(dates_unique_str == start_date_str), "Datetime"]
start_date = start_date.iloc[0]
df_gar_select = df_gar.loc[df_gar["Datetime"] >= start_date, :].copy()

df_gar_select.loc[:, "DateDO"] = df_gar_select["Datetime"].apply(
    lambda x: datetime.datetime.isoformat(x) + ".000+02:00"
)
df_gar_select.loc[:, "Avg HR"] = [
    int(item) if item != "--" else np.nan for item in df_gar_select["Avg HR"]
]
df_gar_select.loc[:, "Max HR"] = [
    int(item) if item != "--" else np.nan for item in df_gar_select["Max HR"]
]

df_SL_select = df_SL.loc[
    [date_SL.date() >= start_date.date() for date_SL in df_SL["Datetime"]], :
]
df_SL_select.loc[:, "DateDO"] = df_SL_select["Datetime"].apply(
    lambda x: datetime.datetime.isoformat(x) + ".000+02:00"
)

In [ ]:
date_DO_gar = np.array(df_gar_select["DateDO"])
text_DO_gar = []

count = 0
for i, date in enumerate(date_DO_gar):

    if df_gar_select["Activity Type"][df_gar_select["DateDO"] == date].values in [
        "Running",
        "Hiking",
        "Walking",
        "Treadmill Running",
    ]:
        row = df_gar_select[df_gar_select["DateDO"] == date]

        activity_str = "{}: {} \n{} km in {}, average pace: {} min/km, max. pace: {} min/km\nAverage HR: {} bpm, max. HR: {} bpm \nAscent: {} m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            [row["Time"].values[0] if "Time" in row else row["Total Time"].values[0]][
                0
            ],
            [
                (
                    row["Avg Speed"].values[0]
                    if "Avg Speed" in row
                    else row["Avg Pace"].values[0]
                )
            ][0],
            [
                (
                    row["Max Speed"].values[0]
                    if "Max Speed" in row
                    else row["Best Pace"].values[0]
                )
            ][0],
            (
                int(row["Avg HR"].values[0])
                if not np.isnan(row["Avg HR"].values[0])
                else "-"
            ),
            (
                int(row["Max HR"].values[0])
                if not np.isnan(row["Max HR"].values[0])
                else "-"
            ),
            [
                (
                    row["Ascent"].values[0]
                    if "Ascent" in row
                    else row["Total Ascent"].values[0]
                )
            ][0],
        )

        text_DO_gar.append(activity_str)
    elif df_gar_select["Activity Type"][df_gar_select["DateDO"] == date].values in [
        "Cycling",
        "Mountain Biking",
    ]:
        row = df_gar_select[df_gar_select["DateDO"] == date]

        activity_str = "{}: {} \n{} km in {}, average speed: {} km/h, max. speed: {} km/h\nAverage HR: {} bpm, max. HR: {} bpm \nTotal ascent: {} m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            [row["Time"].values[0] if "Time" in row else row["Total Time"].values[0]][
                0
            ],
            row["Avg Pace"].values[0],
            row["Best Pace"].values[0],
            (
                int(row["Avg HR"].values[0])
                if not np.isnan(row["Avg HR"].values[0])
                else "-"
            ),
            (
                int(row["Max HR"].values[0])
                if not np.isnan(row["Max HR"].values[0])
                else "-"
            ),
            [
                (
                    row["Ascent"].values[0]
                    if "Ascent" in row
                    else row["Total Ascent"].values[0]
                )
            ][0],
        )

        text_DO_gar.append(activity_str)
    elif df_gar_select["Activity Type"][df_gar_select["DateDO"] == date].values in [
        "Open Water Swimming",
        "Pool Swim",
    ]:
        row = df_gar_select[df_gar_select["DateDO"] == date]

        activity_str = "{}: {} \n{} m in {}, average speed: {} min/100m, max. speed: {} min/100m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            row["Time"].values[0],
            row["Avg Pace"].values[0],
            row["Best Pace"].values[0],
        )

        text_DO_gar.append(activity_str)
    else:
        row = df_gar_select[df_gar_select["DateDO"] == date]

        activity_str = (
            "{}: {} \nDuration: {}\nAverage HR: {} bpm, max. HR: {} bpm ".format(
                row["Activity Type"].values[0],
                row["Title"].values[0],
                row["Time"].values[0],
                (
                    int(row["Avg HR"].values[0])
                    if not np.isnan(row["Avg HR"].values[0])
                    else "-"
                ),
                (
                    int(row["Max HR"].values[0])
                    if not np.isnan(row["Max HR"].values[0])
                    else "-"
                ),
            )
        )

        text_DO_gar.append(activity_str)

df_DO = pd.DataFrame({"date": date_DO_gar, "text": text_DO_gar})

date_DO_SL = np.unique(df_SL_select["DateDO"])

for i, date in enumerate(date_DO_SL):
    df_date = df_SL_select[df_SL_select["DateDO"] == date]
    _, idx = np.unique(df_date["Exercise"], return_index=True)
    sorted_idx = np.sort(idx)
    exercises = df_date["Exercise"].iloc[sorted_idx]

    date_str = ""

    for j, exercise in enumerate(exercises):
        weights = np.unique(df_date["Weight (kg)"][df_date["Exercise"] == exercise])
        if np.isnan(weights[0]):
            weights = "-"
        exercise_str = exercise + " "
        for k, weight in enumerate(weights):
            if k == 0:
                if weight == "-":
                    exercise_str = exercise_str + weight + " kg: "
                else:
                    exercise_str = (
                        exercise_str + str(np.round(weight, decimals=1)) + " kg: "
                    )
            else:
                exercise_str = (
                    exercise_str + " | " + str(np.round(weight, decimals=1)) + " kg: "
                )
            if weights[0] == "-":
                reps = df_date["Reps"][
                    (df_date["Exercise"] == exercise)
                    & (np.isnan(df_date["Weight (kg)"]))
                ]
            else:
                reps = df_date["Reps"][
                    (df_date["Exercise"] == exercise)
                    & (df_date["Weight (kg)"] == weight)
                ]
            _, idx = np.unique(reps, return_index=True)
            sorted_idx = np.sort(idx)
            unique_reps = reps.iloc[sorted_idx]
            for l, unique_rep in enumerate(unique_reps):
                if ~np.isnan(unique_rep):
                    n_sets = np.sum(reps == unique_rep)
                    if l == len(unique_reps) - 1:
                        exercise_str = exercise_str + str(int(n_sets)) + "x" + str(int(unique_rep))
                    else:
                        exercise_str = (
                            exercise_str + str(int(n_sets)) + "x" + str(int(unique_rep)) + ", "
                        )
                else:
                    n_sets = len(reps)
                    exercise_str = exercise_str + str(int(n_sets)) + " sets"

        if j == 0:
            date_str = date_str + exercise_str
        else:
            date_str = date_str + "\n" + exercise_str

    date_str = "\n\nSets and reps:\n" + date_str
    mask_st = df_DO["date"].str.match(date[:10]) & df_DO["text"].str.match(
        "Strength Training"
    )
    mask_b = df_DO["date"].str.match(date[:10]) & df_DO["text"].str.match("Bouldering")

    if len(df_DO.loc[mask_st, "text"].index) != 0:
        for entry in range(len(df_DO.loc[mask_st, "text"].index)):
            target_idx = df_DO.loc[mask_st].index[entry]
            df_DO.loc[target_idx, "text"] = (
                df_DO.loc[mask_st, "text"].iloc[entry] + date_str
            )
    elif len(df_DO.loc[mask_b, "text"].index) != 0:
        for entry in range(len(df_DO.loc[mask_b, "text"].index)):
            target_idx = df_DO.loc[mask_b].index[entry]
            df_DO.loc[target_idx, "text"] = (
                df_DO.loc[mask_b, "text"].iloc[entry] + date_str
            )

### Export to CSV

In [ ]:
csv_name = (
    "GarLeveltoDO_"
    + str(start_date_str).replace("/", "-")
    + "_"
    + str(date_DO_gar[0][:10])
    + ".csv"
)
df_DO.to_csv("Output/" + csv_name, sep=",", header=True, index=False)